In [1]:
import numpy as np
import pandas as pd
import re
import os

In [2]:
from MatrixExamples import m

In [3]:
from AuxFunctions import duplicates_in_list

In [4]:
def zoning(zones: list, names=['O', 'D']) -> pd.MultiIndex:
    '''returns a MultiIndex object with zones for origins and destinations.'''
    
    if all(isinstance(elem, list) for elem in zones):
        ODs = zones
        if not any(duplicates_in_list for lst in zones):
            raise ValueError('There are duplicated zones')
    
    elif isinstance(zones,list):
        ODs = [zones for name in names]
        if duplicates_in_list(zones):
            raise ValueError('There are duplicated zones')
    
    else:
        raise ValueError('"zones" must be a list or a list of lists')
    
    idx = pd.MultiIndex.from_product(ODs, names=names)
    
    if idx.names != names:
        raise ValueError('zoning could not be created from {}.'.format(zones) + 
                         '\n"zones" must be a list or a list of lists')
    
    return idx

In [5]:
class Matrix(pd.DataFrame):
    '''A Matrix in Transport Planning is a pandas DataFrame,
    with Origins and Destinations as MultiIndex levels: [O, D]'''

    @property
    def _constructor(self):
        '''Matrix operations returns Matrix objects.'''
        return Matrix
    
    @property
    def Os(self):
        '''Returns origin names without duplicates.'''
        return list(self.index.get_level_values(0).unique())
    
    @property
    def Ds(self):
        '''Returns destination names without duplicates.'''
        return list(self.index.get_level_values(1).unique())
    
    @property
    def TO(self):
        '''Returns trip-ends for origins.'''
        return self.groupby(level=0).sum()
    
    @property
    def TD(self):
        '''Returns trip-ends for destinations.'''
        return self.groupby(level=1).sum()
    
    @property
    def TOTALS(self):
        '''Returns the matrix totals.'''
        return self.sum()
    
    def TransposeOD(self, sort=True):
        '''Transposes a matrix in ODT format: swaps Origins and Destinations.'''
        mat_T = self.swaplevel()
        mat_T.index.names = self.index.names
        if sort:
            mat_T = mat_T.sort_index()
        return mat_T
    
    @property
    def TOp(self):
        '''Returns origin proportions: Pij = Tij / TOi'''
        return self.groupby(level=0).apply(lambda x: (x/x.sum()).fillna(0))
    
    @property
    def TDp(self):
        '''Returns destination proportions: Pij = Tij / TDj'''
        return self.groupby(level=1).apply(lambda x: (x/x.sum()).fillna(0))
    
    @property
    def matrix(self):
        '''Returns matrix as a tradicional 2D matrix.'''
        return self.to_panel()
    
    #TODO:
    @staticmethod
    def from_panel(panel):
        ...
    
    def rezone(self, mapping: pd.DataFrame, mapping_cols=['old', 'new'],
               weight_cols=None, calculate_proportions=True,
               min_weight=0.00000001):
        '''Changes the zoning system based on mapping.
        A mapping is a correspondence between old zones and new zones.
        
           weights - ['Owght', 'Dwght'] to use for zone disaggregation
           calculate_proportions - if True, weight proportions will be 
               calculated and applied
           min_weight - value for weights with value zero'''
        
        if weight_cols:
            
            if len(weight_cols)!=2:
                raise ValueError("weight_cols must be as in ['Owght', 'Dwght']")
            
            Owght, Dwght = weight_cols
            
            #cap to min_weight
            Omap = mapping[mapping_cols].copy()
            Omap[Owght] = mapping[Owght].where(mapping[Owght] > min_weight, min_weight)
            Dmap = mapping[mapping_cols].copy()
            Dmap[Dwght] = mapping[Dwght].where(mapping[Dwght] > min_weight, min_weight)
            
            if calculate_proportions:
                #proportions always respect 'old' mapping column
                Omap[Owght] = Omap.groupby(mapping_cols[0])[Owght].apply(lambda x: (x/x.sum()))
                Dmap[Dwght] = Dmap.groupby(mapping_cols[0])[Dwght].apply(lambda x: (x/x.sum()))
        else:
            Omap = mapping.reset_index()[mapping_cols]
            Dmap = Omap
        
        suffixes = ['_' + n for n in self.index.names]
        mat = pd.merge(self.reset_index(), Omap.reset_index(),
                      left_on=self.index.names[-2], right_on=mapping_cols[0])
        mat = pd.merge(mat, Dmap.reset_index(),
                      left_on=self.index.names[-1], right_on=mapping_cols[0],
                      suffixes=suffixes)
        
        if weight_cols:
            if Owght == Dwght:
                Owght, Dwght = ['{}{}'.format(Owght,s) for s in suffixes]
                
            for col in self:
                mat[col] = mat[col] * mat[Owght] * mat[Dwght]
        
        NewODnames = ['{}{}'.format(mapping_cols[1],s) for s in suffixes]
        
        aux_cols = list(set(mat.columns) - set(self.columns) - set(NewODnames))
        mat = mat.drop(aux_cols, axis=1)
        
        mat = mat.groupby(NewODnames).sum()
        mat = Matrix(mat)
        
        if not all(x==y for x,y in zip(self.TOTALS, mat.TOTALS)):
            print("WARNING: rezoned matrix does not preserve the matrix totals.")
        
        return mat
    
        #TODO: update this function to allow outputing a weighted average (eg: for costs)
        # Deal with 0 trips in wght file:
            ## 1) Multiply src_file by wght_file
            ## 2) Disaggregate src x wght as if it was a demand matrix
            ## 3) Disaggregate wght as a demmand matrix as well
            ## 4) merge src_wght_hyl and wght_hyl on the first two columns:
            ## 5) Divide src x wght / wght at hyl level
            ## Aggregate!
        
    def complete(self, zones):
        '''Rompletes the matrix index with specified zones. Ignores existing zones.'''
        if isinstance(zones, pd.MultiIndex):
            #zones is a zoning system already (MultiIndex)
            zoning = zones
        elif isinstance(zones, list):
            #zones is just a list that needs to be expanded
            zoning = zoning(zones)
        else:
            raise ValueError('"zones" must be list of zones or zoning system (MultiIndex)')
        zoning_union = self.index.union(zoning)
        return self.reindex(index=zoning_union)
    
    def submatrix(self, zoning: pd.MultiIndex):
        '''Returns a submatrix with the origins and destinations specified in zoning'''
        zoning_intersect = self.index.intersection(zoning)
        return self.reindex(zoning_intersect)
    
    @staticmethod
    def read_EMME(file):
        '''Reads a text file containing one or more matrices in EMME format.
        Accepts matrices, trip origins, trip destinations and constants.
        Assumes one single value per row in the EMME files.
        '''

        EMMErecord_cols = {
            'md': ['zone', '_TD'],
            'mo': ['zone', '_TO'],
            'mf': ['O', 'D', '']
            } #TODO: remove difference by TO /TD / T ??

        ## RegEx to read EMME format:
        mat_re = re.compile(r'a matrix\s*=\s*(mo|md|mf|ms)(\d+)\s+(\w+?)\s+(-?\d+)\s+(.+?)\n(.*)\n(?=a matrix|d matrix|\Z|\s*\n)',
                           re.DOTALL | re.MULTILINE)
        # re groups:
        # mat_type, mat_num, mat_name, mat_default, mat_desc, mat_data

        EMMErecord_re = {
            'md': re.compile(r'\s*all\s+(\d+)\s*:\s*(-?\.?\d+\.?\d*)\n'),
            'mo': re.compile(r'\s*(\d+)\s+all\s*:\s*(-?\.?\d+\.?\d*)\n'),
            'mf': re.compile(r'\s*(\d+)\s+(\d+)\s*:\s*(-?\.?\d+\.?\d*)\n')
            } #TODO: implement ms

        ## Read Data
        data = {}
        fn, fext = os.path.splitext(file)
        data[fn] = {}
        with open(file, 'r') as f:
            fcontent = f.read()
            #each source file might contain several matrices
            mat_blocks = mat_re.findall(fcontent)
            for matb in mat_blocks:
                mat_type, mat_num, mat_name, mat_default, mat_desc, mat_data = matb
                mat_rows = EMMErecord_re[mat_type].findall(mat_data)
                #TODO: Use named tuples
                data[fn][mat_name] = dict(zip(
                   'mat_type, mat_num, mat_default, mat_desc, mat_rows'.split(', '),
                   [mat_type, mat_num, mat_default, mat_desc, mat_rows]))

        ## Convert to DataFrame
        matrix = Matrix()
        for fn in data:
            for matn in data[fn]:
                mat_data = data[fn][matn]

                #convert rows into df, setting column names and index
                df_cols = EMMErecord_cols[mat_data['mat_type']]
                df_idx_cols = df_cols[:-1]
                df_data_cols = df_cols[-1]

                mat = Matrix.from_records(mat_data['mat_rows'],
                                               columns=df_cols,
                                               index=df_idx_cols)

                #this avoids repeating names:
                mat_id = '{}{}'.format(fn, df_data_cols)
                #numeric is needed for Matrix methods to work as expected
                matrix[mat_id] = pd.to_numeric(mat[df_data_cols])           

        return matrix
        
    #TODO:
    def join_orig(df):
        '''Joins df on the matrix origins.'''
        ...
        
    #TODO:
    def join_dest(df):
        '''Joins df on the matrix destinations.'''
        ...
    
    def furness(self, TO, TD, tolerance=0.001, max_iter=100):
        '''Use FRATAR algorithm to adjust (balance) the matrix
        to target origins and destinations (TO, TD), within a certain tolerance.
        Will not always converge, hence cap maximum iterations to max_iter.'''
        
        sTO = self.TO
        sTD = self.TD
        fmat = self.copy()
        i = 1
        
        while True:
            # Balancing factors (Nan ~> 0, inf ~> 1):
            A = (TO / fmat.TO).fillna(0).replace([np.inf, -np.inf], 1)
            B = (TD / fmat.TD).fillna(0).replace([np.inf, -np.inf], 1)
            fmat = fmat.mul(A, axis=1, level=0).mul(B, axis=1, level=1)
        
            i += 1
            within_tol = not (any((fmat.TO - TO) > tolerance) or any((fmat.TD - TD) > tolerance))
            
            if within_tol:
                break
            if max_iter and (i >= max_iter):
                break
        
        return fmat

In [6]:
mat = Matrix(m)
mat

T1  T2  T3
O D            
1 1   0   1   0
  2   1   2   6
  3   1   3  10
  4   0   4  12
  5   1   5  12
  6   1   6  10
  7   0   7   6
2 1   1   8   0
  2   1   9   0
  3   0  10  10
  4   1  11  12
  5   1  12  12
  6   0  13  10
  7   1  14   6
3 1   1  15   0
  2   0  16   6
  3   1  17   0
  4   1  18  12
  5   0  19  12
  6   1  20  10
  7   1  21   6
4 1   0  22   0
  2   1  23   6
  3   1  24  10
  4   0  25   0
  5   1  26  12
  6   1  27  10
  7   0  28   6
5 1   1  29   0
  2   1  30   6
  3   0  31  10
  4   1  32  12
  5   1  33   0
  6   0  34  10
  7   1  35   6
6 1   1  36   0
  2   0  37   6
  3   1  38  10
  4   1  39  12
  5   0  40  12
  6   1  41   0
  7   1  42   6
7 1   0  43   0
  2   1  44   6
  3   1  45  10
  4   0  46  12
  5   1  47  12
  6   1  48  10
  7   0  49   0

In [7]:
zoning1 = zoning(list(range(3)))
zoning2 = zoning(list(range(10)))
zoning3 = zoning([5+i for i in range(5)])
zoning4 = zoning('A B C'.split())

In [8]:
mat.complete(zoning2)

T1    T2    T3
O D                 
0 0  NaN   NaN   NaN
  1  NaN   NaN   NaN
  2  NaN   NaN   NaN
  3  NaN   NaN   NaN
  4  NaN   NaN   NaN
  5  NaN   NaN   NaN
  6  NaN   NaN   NaN
  7  NaN   NaN   NaN
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN
1 0  NaN   NaN   NaN
  1  0.0   1.0   0.0
  2  1.0   2.0   6.0
  3  1.0   3.0  10.0
  4  0.0   4.0  12.0
  5  1.0   5.0  12.0
  6  1.0   6.0  10.0
  7  0.0   7.0   6.0
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN
2 0  NaN   NaN   NaN
  1  1.0   8.0   0.0
  2  1.0   9.0   0.0
  3  0.0  10.0  10.0
  4  1.0  11.0  12.0
  5  1.0  12.0  12.0
  6  0.0  13.0  10.0
  7  1.0  14.0   6.0
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN
..   ...   ...   ...
7 0  NaN   NaN   NaN
  1  0.0  43.0   0.0
  2  1.0  44.0   6.0
  3  1.0  45.0  10.0
  4  0.0  46.0  12.0
  5  1.0  47.0  12.0
  6  1.0  48.0  10.0
  7  0.0  49.0   0.0
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN
8 0  NaN   NaN   NaN
  1  NaN   NaN   NaN
  2  NaN   NaN   NaN
  3  NaN   NaN   NaN
  4  NaN   NaN   NaN
  5  NaN   NaN   NaN
  6  NaN   NaN   NaN
  7  NaN   NaN   NaN
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN
9 0  NaN   NaN   NaN
  1  NaN   NaN   NaN
  2  NaN   NaN   NaN
  3  NaN   NaN   NaN
  4  NaN   NaN   NaN
  5  NaN   NaN   NaN
  6  NaN   NaN   NaN
  7  NaN   NaN   NaN
  8  NaN   NaN   NaN
  9  NaN   NaN   NaN

[100 rows x 3 columns]

In [9]:
mat.matrix.T3

D,1,2,3,4,5,6,7
O,,,,,,,
1,0,6,10,12,12,10,6
2,0,0,10,12,12,10,6
3,0,6,0,12,12,10,6
4,0,6,10,0,12,10,6
5,0,6,10,12,0,10,6
6,0,6,10,12,12,0,6
7,0,6,10,12,12,10,0


In [10]:
mat.TDp.matrix.T3

D,1,2,3,4,5,6,7
O,,,,,,,
1,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
2,0.0,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667
3,0.0,0.166667,0.000000,0.166667,0.166667,0.166667,0.166667
4,0.0,0.166667,0.166667,0.000000,0.166667,0.166667,0.166667
5,0.0,0.166667,0.166667,0.166667,0.000000,0.166667,0.166667
6,0.0,0.166667,0.166667,0.166667,0.166667,0.000000,0.166667
7,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000


In [11]:
basicmapping = pd.DataFrame({
        'sectors': 'A A B B B C C'.split(),
        'zones':   [1,2,3,4,5,6,7,]
    })

mapping = pd.DataFrame({
        'sectors': 'A B A B B B C C C'.split(),
        'zones':   [1,2,2,3,4,5,6,7,5],
        'Val1':    [1,4,4,2,1,3,1,4,2],
        'Val2':    [3,0.01,1,2,1,3,3,1,0.01]
    })

In [12]:
mat.rezone(basicmapping, ['zones', 'sectors'])

T1   T2  T3
sectors_O sectors_D             
A         A           3   20   6
          B           4   45  68
          C           2   40  32
B         A           4  135  18
          B           6  225  68
          C           4  165  48
C         A           2  160  12
          B           4  255  68
          C           3  180  16

In [13]:
rezoned = mat.rezone(mapping, ['zones', 'sectors'], weight_cols=['Val1', 'Val2'])
rezoned

T1          T2         T3
sectors_O sectors_D                                 
A         A          1.985149   11.435644   5.940594
          B          3.009868   28.527812  50.999605
          C          1.504983   26.536545  24.059801
B         A          4.179208  119.291089  15.445545
          B          6.213815  203.473695  76.254788
          C          4.106977  151.135216  49.699668
C         A          2.786139  182.679208  14.257426
          B          4.809210  293.987902  76.862840
          C          3.404651  207.932890  22.479734

In [14]:
rezoned.TOTALS

T1      32.0
T2    1225.0
T3     336.0
dtype: float64

In [15]:
mat.TOTALS

T1      32
T2    1225
T3     336
dtype: int64

In [16]:
DEMMAND = Matrix.read_EMME('Demand_EMME.txt')
DEMMAND

Demand_EMME
O    D                
101  101        1.5615
     102        0.0338
     103        4.1628
     104        2.0070
     105        1.5930
     106        0.1395
     107        0.8108
     108        0.7475
     109        0.2345
     110        0.3509
     111        0.6626
     112        0.4458
     113        0.3684
     114        0.5735
     115        0.0944
     116        0.2403
     117        0.3339
     118        0.2622
     119        0.2569
     201        0.1164
     202        0.0737
     203        0.1196
     204        0.2698
     205        0.0237
     206        0.2613
     207        0.0106
     208        0.0095
     209        0.1387
     210        0.0972
     211        0.0698
...                ...
1913 1780       0.4074
     1781       0.0858
     1782       1.3428
     1783       0.7599
     1784       0.0816
     1801    3155.7934
     1802     119.7499
     1803    1188.3122
     1804      28.7936
     1805      71.6565
     1806       6.2300
     1807      26.4169
     1808       3.7560
     1809      11.4795
     1810      66.1949
     1811       1.5953
     1812      34.7306
     1813       1.8099
     1901       2.3164
     1902      11.2327
     1903      45.7226
     1904    5741.9848
     1905      29.2176
     1906      14.8447
     1907       3.6403
     1908       5.7690
     1909      84.8270
     1910       1.7470
     1911      15.4561
     1912     557.1537

[707280 rows x 1 columns]

In [17]:
DIST = Matrix.read_EMME('Dist_EMME.txt')
DIST

Dist_EMME
O   D                
1   1         0.00000
    2      7182.00000
    3     15634.99800
    4     23180.99800
    5     15533.99800
    6     30531.00390
    7     47291.00000
    8      9940.00195
    9     10772.00290
    10    17327.00190
    11    26253.00000
    12    45012.00780
    13    53452.00780
    14    51295.01170
    15    90100.00000
    16    91504.98430
    17    60492.00000
    18    68650.00000
    19    69266.00780
    20    67486.00000
    21    49757.00780
    22    34456.00000
    23    51594.00000
    24   102760.98400
    25    64037.98820
    26    41513.00000
    27    91613.00000
    28   108692.00000
    29    73069.63280
    30    65341.63670
...               ...
149 119   12711.98310
    120   15081.77390
    121   13096.00980
    122   16533.95410
    123   16509.12840
    124   17529.24740
    125   13382.89200
    126   27195.85680
    127   31722.18240
    128   25792.68790
    129   25349.91560
    130   23291.95230
    131   25776.55050
    132   41845.24460
    133   36035.65900
    134   42309.50000
    135   37355.94660
    136   51069.27820
    137   50770.79190
    138   47824.90030
    139   55480.78080
    140   45454.57720
    141   44222.90500
    142   41739.51850
    143   38885.96880
    144   31699.49650
    145   21592.85070
    146   18044.11030
    147   17055.16570
    148   10935.56100

[22137 rows x 1 columns]

In [18]:
import random

def randomizeSeries(S, fraction):
    '''Adds variability to a pd.Series'''
    return S+pd.Series([random.randint(-int(x/fraction),int(x/fraction)) for x in S], index=S.index)

def randomizeTE(TE):
    return TE.apply(randomizeSeries, args=[10])

In [19]:
tTO = randomizeTE(mat.TO)
tTO

,T1,T2,T3
O,,,
1,4,30,54
2,5,79,48
3,5,123,45
4,4,185,43
5,5,224,47
6,5,266,44
7,4,330,55


In [20]:
tTD = randomizeTE(mat.TD)
tTD

,T1,T2,T3
D,,,
1,4,153,0
2,5,161,36
3,5,170,61
4,4,166,75
5,5,177,75
6,5,198,58
7,4,198,38


In [21]:
fmat = mat.furness(tTO, tTD)
fmat

T1         T2         T3
O D                           
1 1  0.0   1.062588   0.000000
  2  1.0   2.138199   5.736540
  3  1.0   3.244276   9.751831
  4  0.0   4.053578  12.000430
  5  1.0   5.193301  12.195080
  6  1.0   6.711190   9.222209
  7  0.0   7.548044   6.209299
2 1  1.0   8.157958   0.000000
  2  1.0   9.233944   0.000000
  3  0.0  10.378225   9.675325
  4  1.0  10.697882  11.906283
  5  1.0  11.961380  12.099406
  6  0.0  13.954626   9.149858
  7  1.0  14.487418   6.160585
3 1  1.0  14.561064   0.000000
  2  0.0  15.626981   5.808106
  3  1.0  16.795090   0.000000
  4  1.0  16.664334  12.150141
  5  0.0  18.028683  12.347220
  6  1.0  20.436908   9.337261
  7  1.0  20.686766   6.286763
4 1  0.0  23.132205   0.000000
  2  1.0  24.331866   5.839419
  3  1.0  25.682492   9.926720
  4  0.0  25.069633   0.000000
  5  1.0  26.722449  12.413786
  6  1.0  29.884188   9.387600
  7  0.0  29.876094   6.320656
5 1  1.0  28.847700   0.000000
  2  1.0  30.025324   6.411567
  3  0.0  31.383868  10.899343
  4  1.0  30.358254  13.412537
  5  1.0  32.087486   0.000000
  6  0.0  35.602088  10.307400
  7  1.0  35.330736   6.939955
6 1  1.0  34.895415   0.000000
  2  0.0  36.084512   5.613499
  3  1.0  37.487030   9.542668
  4  1.0  36.053223  11.743038
  5  0.0  37.899582  11.933513
  6  1.0  41.834355   0.000000
  7  1.0  41.312988   6.076118
7 1  0.0  43.842654   0.000000
  2  1.0  45.137167   6.584592
  3  1.0  46.695224  11.193478
  4  0.0  44.730096  13.774495
  5  1.0  46.841931  13.997920
  6  1.0  51.517284  10.585560
  7  0.0  50.698593   0.000000

In [22]:
max([max(x,y) for x,y in zip((fmat.TO - tTO).abs().max(), (fmat.TD - tTD).abs().max())])

1.9406385326620921